In [39]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.python.framework.ops import disable_eager_execution, enable_eager_execution
disable_eager_execution()

import os
import sys
sys.path.insert(0, '/mnt/home/raheppt1/projects/age_prediction')
import numpy as np
import datetime
from pathlib import Path

# tensorflow-gpu 2.1.0
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, Conv2D
from tensorflow.keras.layers import BatchNormalization, ReLU
from tensorflow.keras.optimizers import *
import tensorflow_probability as tfp

from dataset import AgeData
from misc import utils 

In [40]:
print(tf.__version__)

2.1.0


In [41]:
# Define GPU device.
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="2";

# Activate memory growth.
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

## Parameters

In [42]:
# General parameters.
batch_size = 8
image_size = [100, 120, 100]
image_spacing = [1.5, 1.5, 1.5]

logroot_dir = Path('../logs/keras/')
checkpoint_dir = Path('/mnt/share/raheppt1/tf_models/age/keras')

### Datasets

In [61]:
# Load training and validation data.
# todo select split, ADNI/IXI
age_data = AgeData(image_size,
                   image_spacing,
                   shuffle_training_images=True,
                   save_debug_images=False)

dataset_train = age_data.dataset_train()
train_samples = dataset_train.num_entries()
dataset_val = age_data.dataset_val()
val_samples = dataset_val.num_entries()
print(train_samples, val_samples)

# Define training and validation datasets from generators.
def train_gen():
    data = dataset_train
    i = 0
    while i < data.num_entries():
        sample = data.get_next()
        # DHWC tensor format
        image = sample['generators']['image'].transpose([1, 2, 3, 0])
        image = image.astype('float32')
        age = sample['generators']['age']
        yield image, age
        i += 1

def val_gen():
    data = dataset_val
    i = 0
    while i < data.num_entries():
        sample = data.get_next()
        image = sample['generators']['image'].transpose([1, 2, 3, 0])
        image = image.astype('float32')
        age = sample['generators']['age']
        yield image, age
        i += 1

ds_train = tf.data.Dataset.from_generator(train_gen, 
                                          output_types=(tf.float32, tf.float32),
                                          output_shapes=(tf.TensorShape((None, None, None, None)), 
                                                         tf.TensorShape((1, ))))
ds_train = ds_train.batch(batch_size=batch_size)

ds_val = tf.data.Dataset.from_generator(val_gen, 
                                        output_types=(tf.float32, tf.float32),
                                        output_shapes=(tf.TensorShape((None, None, None, None)), 
                                                         tf.TensorShape((1, ))))
ds_val = ds_val.batch(batch_size=batch_size)

loaded 410 ids
loaded 103 ids
410 103


In [44]:
# Check data.

## Discriminative model

In [45]:
def build_discriminative_model(input_shape=(124, 124, 124, 1), 
                               lambda_l2=0.00005):
    model = Sequential()
    model.add(Conv3D(8, kernel_size=(3, 3, 3),  padding='same', input_shape=input_shape))
    model.add(ReLU())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Conv3D(16, kernel_size=(3, 3, 3), padding='same'))
    model.add(ReLU())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
    model.add(ReLU())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
    model.add(ReLU())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Conv3D(128, kernel_size=(3, 3, 3), padding='same'))
    model.add(ReLU())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu',
                    kernel_regularizer=tf.keras.regularizers.l2(lambda_l2)))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu',
                    kernel_regularizer=tf.keras.regularizers.l2(lambda_l2)))
    model.add(Dense(1, activation='linear', bias_initializer=tf.constant_initializer(50.0)))
    return model

In [47]:
# Training parameters.
max_epochs = 3000
learning_rate = 0.0001
lambda_l2 = 0.00005
run_name = 'disc_01'
beta_1 = 0.9
beta_2 = 0.999

# Initialize tensorboard logdir.
logdir = logroot_dir.joinpath(datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + run_name)
logdir = str(logdir)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Save checkpoints. Early stopping, only save the best checkpoint.
checkpoint_path = checkpoint_dir.joinpath(run_name,'cp.ckt')
checkpoint_path = str(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_best_only=True,
                                                 verbose=1)

disc_model = build_discriminative_model(image_size + [1], lambda_l2)
disc_model.compile(loss='mse', 
                   optimizer=Adam(learning_rate,
                                  beta_1 = beta_1,
                                  beta_2 = beta_2), 
                   metrics=['mse', 'mae'])
disc_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_10 (Conv3D)           (None, 100, 120, 100, 8)  224       
_________________________________________________________________
re_lu_40 (ReLU)              (None, 100, 120, 100, 8)  0         
_________________________________________________________________
max_pooling3d_40 (MaxPooling (None, 50, 60, 50, 8)     0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 50, 60, 50, 16)    3472      
_________________________________________________________________
re_lu_41 (ReLU)              (None, 50, 60, 50, 16)    0         
_________________________________________________________________
max_pooling3d_41 (MaxPooling (None, 25, 30, 25, 16)    0         
_________________________________________________________________
conv3d_12 (Conv3D)           (None, 25, 30, 25, 32)  

In [ ]:
# Train the model.
hist_discriminative = disc_model.fit(ds_train, 
                                     epochs=max_epochs, 
                                     verbose=1,
                                     validation_data=ds_val,
                                     callbacks=[tensorboard_callback,
                                                cp_callback])

### Probabilistic model

In [49]:
# todo test full probabilistic model
def build_probmodel(input_shape=(124, 124, 124, 1)):
    model = Sequential()
    model.add(tfp.layers.Convolution3DReparameterization(8, kernel_size=(3, 3, 3),  padding='same', input_shape=input_shape))
    model.add(ReLU())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(tfp.layers.Convolution3DReparameterization(16, kernel_size=(3, 3, 3), padding='same'))
    model.add(ReLU())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(tfp.layers.Convolution3DReparameterization(32, kernel_size=(3, 3, 3), padding='same'))
    model.add(ReLU())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(tfp.layers.Convolution3DReparameterization(64, kernel_size=(3, 3, 3), padding='same'))
    model.add(ReLU())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(tfp.layers.Convolution3DReparameterization(128, kernel_size=(3, 3, 3), padding='same'))
    model.add(ReLU())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Flatten())
    model.add(tfp.layers.DenseReparameterization(1024, activation='relu'))
    model.add(tfp.layers.DenseReparameterization(512, activation='relu'))
    model.add(tfp.layers.DenseReparameterization(2, activation='linear'))
    model.add(tfp.layers.DistributionLambda(
              lambda t: tfp.distributions.Normal(loc=t[..., :1],
                                                 scale=1e-3 + tf.math.softplus(0.05 * t[..., 1:]))))
    #model.add(tfp.layers.DistributionLambda(lambda t: tfp.distributions.Independent(
    #                                        tfp.distributions.Normal(loc=t, scale=1),
    #                                        reinterpreted_batch_ndims=1)))
    return model

### Settings

In [50]:
# Image parameters
image_size = [100, 120, 100]
image_spacing = [1.5, 1.5, 1.5]

# Parameters:
batch_size = 8 # 16 8
max_epochs = 2000
# Adam
learning_rate = 0.0001# 0.0001

In [51]:
def neg_log_likelihood(y_true, y_pred):
    neg_log_like = -y_pred.log_prob(y_true)
    neg_log_like = tf.reduce_mean(input_tensor=neg_log_like)
    return neg_log_like

In [69]:
# Training parameters.
max_epochs = 3000
lr=0.001
run_name = 'prob'
beta_1 = 0.9
beta_2 = 0.999

# Initialize tensorboard logdir.
logdir = logroot_dir.joinpath(datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + run_name)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=str(logdir))

# Save checkpoints. Early stopping, only save the best checkpoint.
checkpoint_path = checkpoint_dir.joinpath(run_name,'cp.ckt')
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=str(checkpoint_path),
                                                 save_weights_only=True,
                                                 save_best_only=True,
                                                 verbose=1)

bcnn_model = build_probmodel(image_size + [1])
loss_fnc = create_loss(bcnn_model)
bcnn_model.compile(loss=lambda t, y: batch_size*neg_log_likelihood(t, y), 
                   optimizer=Adam(learning_rate,
                                  beta_1=beta_1,
                                  beta_2=beta_2), 
                   metrics=['mse', 'mae'])
print("BCNN Model:")
bcnn_model.summary()

BCNN Model:
Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_reparameterization_50 (None, 100, 120, 100, 8)  440       
_________________________________________________________________
re_lu_65 (ReLU)              (None, 100, 120, 100, 8)  0         
_________________________________________________________________
max_pooling3d_65 (MaxPooling (None, 50, 60, 50, 8)     0         
_________________________________________________________________
conv3d_reparameterization_51 (None, 50, 60, 50, 16)    6928      
_________________________________________________________________
re_lu_66 (ReLU)              (None, 50, 60, 50, 16)    0         
_________________________________________________________________
max_pooling3d_66 (MaxPooling (None, 25, 30, 25, 16)    0         
_________________________________________________________________
conv3d_reparameterization_52 (None, 25, 3

In [70]:
n_epochs = 1
# todo check loss function in regression example
history = bcnn_model.fit(ds_train, 
                         epochs=n_epochs,
                         steps_per_epoch=2,#train_samples//batch_size,
                         validation_steps=1,#val_samples//batch_size,
                         verbose=1,
                         validation_data=ds_val,
                         callbacks=[tensorboard_callback,
                                    cp_callback])

Train on 2 steps, validate on 1 steps
1/2 [==============>...............] - ETA: 16s - loss: 70485471657984.0000 - mse: 2202671.0000 - mae: 1473.7245
Epoch 00001: val_loss improved from inf to 1021412507648.00000, saving model to /mnt/share/raheppt1/tf_models/age/keras/prob/cp.ckt
2/2 [==============================] - 36s 18s/step - loss: 35561685024768.0000 - mse: 1111302.5000 - mae: 797.3650 - val_loss: 1021412507648.0000 - val_mse: 31918.8047 - val_mae: 157.4292


In [60]:
history

In [ ]:
bcnn_model.predict(imgs)

In [ ]:
# todo predict
# todo quantile regression
# todo normal output layer
# mae for probabilistic output 